In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn import metrics

In [2]:
#Data-Cleaning Function: Fill in the missing values for the relevant fields of the classification problem and 
#adds combination of attributes to the dataframe for feature extraction 
def clean_data(data_frame):
    data_frame['Ins_Age'] = data_frame['Ins_Age'].fillna(data_frame['Ins_Age'].mean())
    data_frame['Wt'] = data_frame['Wt'].fillna(data_frame['Wt'].mean())
    data_frame['Product_Info_2'] = pd.factorize(data_frame['Product_Info_2'])[0]
    data_frame['BMI'] = data_frame['BMI'].fillna(1.17669 * data_frame['Wt'] + 0.125572)
    data_frame['BMI_Age'] = data_frame['BMI'] * data_frame['Ins_Age']
    data_frame['prop_BMI_Wt'] = ((data_frame['BMI'] * data_frame['Wt'])** 2)
    return data_frame

In [3]:
#Data-frame Construction: from training dataset 
dataframe_training = pd.read_csv("training.csv")
dataframe_training["Product_Info_2"] = pd.factorize(dataframe_training["Product_Info_2"])[0] #Encoding alphanumerical 
                                                                                        #fields to numerical fields.

In [4]:
dataframe_training.describe()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
count,20000.000000,19016.000000,20000.000000,18999.000000,19016.000000,19049.000000,19009.000000,18945.000000,19022.000000,19032.000000,...,18951.000000,18992.000000,19013.000000,19020.000000,18997.000000,18978.000000,19006.000000,19008.000000,19034.000000,20000.000000
mean,9999.500000,1.027398,3.692900,24.431917,0.327652,2.006667,2.672787,1.042491,0.404401,0.707558,...,0.056461,0.010267,0.047283,0.010358,0.007896,0.013226,0.007997,0.019729,0.054324,5.624250
std,5773.647028,0.163244,3.842919,5.061490,0.282560,0.081381,0.739856,0.288321,0.195850,0.074419,...,0.230817,0.100810,0.212250,0.101246,0.088510,0.114244,0.089073,0.139070,0.226662,2.449094
min,0.000000,1.000000,-1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4999.750000,1.000000,1.000000,26.000000,0.076923,2.000000,3.000000,1.000000,0.238806,0.654545,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,9999.500000,1.000000,2.000000,26.000000,0.230769,2.000000,3.000000,1.000000,0.402985,0.709091,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,14999.250000,1.000000,6.000000,26.000000,0.487179,2.000000,3.000000,1.000000,0.567164,0.763636,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
max,19999.000000,2.000000,18.000000,37.000000,1.000000,3.000000,3.000000,3.000000,0.925373,0.963636,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000


In [5]:

dataframe_training = clean_data(dataframe_training) #cleaning the dataframe 
prediction_variable = dataframe_training['Response'] #Assigning the prediction variable for the classifier
dataframe_training = dataframe_training.drop(['Id','Response'], axis = 1) #Dropping the ID and Response columns from the dataframe
dataframe_training.fillna(-1, inplace=True)

In [6]:
dataframe_training.describe()

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,...,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,BMI_Age,prop_BMI_Wt
count,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.927650,4.692900,23.15905,0.262331,1.863700,2.490800,0.934750,0.404401,0.624912,0.293242,...,-0.040650,-0.004400,-0.039150,-0.042650,-0.038550,-0.042100,-0.030850,0.003400,0.193287,0.028329
std,0.466504,3.842919,7.42221,0.397960,0.644781,1.074976,0.535917,0.191002,0.373589,0.086707,...,0.241869,0.307059,0.239416,0.236292,0.249333,0.235649,0.259618,0.316217,0.108512,0.038830
min,-1.000000,0.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.058577,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000123
25%,1.000000,2.000000,26.00000,0.076923,2.000000,3.000000,1.000000,0.253731,0.636364,0.234310,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108315,0.008511
50%,1.000000,3.000000,26.00000,0.230769,2.000000,3.000000,1.000000,0.404401,0.709091,0.288703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.183207,0.016939
75%,1.000000,7.000000,26.00000,0.487179,2.000000,3.000000,1.000000,0.552239,0.763636,0.341004,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.264142,0.032277
max,2.000000,19.000000,37.00000,1.000000,3.000000,3.000000,3.000000,0.925373,0.963636,0.801255,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.805970,0.642010


In [7]:
#Extraction of the training and testing sets for the classifier using train_test_split method from sklearn library
data_train, data_test, response_train, response_test = train_test_split(dataframe_training, prediction_variable, test_size = 0.1, random_state = 0)

In [8]:
#Initializing the XGBClassifier as classifier_xgb with tuned parameters
classifier_xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [9]:
#Running the XGB Classifier on Training data
classifier_xgb.fit(data_train, response_train, eval_metric = 'auc')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=6, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [10]:
#Predicting the results for the Testing data
prediction_xgb = classifier_xgb.predict(data_test)

In [11]:
#Comparing the results of the classifier prediction with actual reponse variable
metrics.accuracy_score(response_test, prediction_xgb)

0.56999999999999995

In [12]:
#Testing Dataframe Contruction and cleaning
dataframe_testing = pd.read_csv("testing.csv")
main_solution = pd.DataFrame(dataframe_testing['Id']) #Exracting the ID column for the final output dataframe
dataframe_testing = dataframe_testing.drop(['Id'], axis = 1)
dataframe_testing = clean_data(dataframe_testing)
dataframe_testing.fillna(-1, inplace=True)

In [13]:
#Classifier Predciton for the Testing Dataset
output_prediction = classifier_xgb.predict(dataframe_testing)

In [14]:
#Appending the results to the output file
main_solution['Response'] = output_prediction.tolist()
main_solution.to_csv('insurance_prediction_xgb.csv', index = False)

In [15]:
output_prediction

array([8, 8, 8, ..., 8, 8, 6])